# ETL B3

#### Importação de dependências python

In [1]:
from assets.bot import get_b3_data
import pandas as pd

#### Extrair ações da bolsa de valores B3
Extrair dados atualizados do site "fundamentus.com.br" em background fazendo uso da biblioteca Selenium e armazenar os dados extraídos em um Dataframe Pandas.

In [2]:
df = get_b3_data()

#### Tratar dados dados e convertê-los para numérico
Tratar os dados para tornar possível a conversão de string para float

In [3]:
columns_to_convert = df.columns[1:]
for c in columns_to_convert:
    df[c] = df[c].str.replace('.','')
    df[c] = df[c].str.replace(',','.')
    df[c] = df[c].str.replace('%','')
    df[c] = pd.to_numeric(df[c], 'coerce')

#### Filtrar dados extraídos por P/L inferiores a 10 e superiores a 3
Efetuar a filtragem dos dados por P/L (Preço sobre Lucro) menores que 10 e maiores que 3 para evitar distorções.

In [4]:
# Filtrar linhas por colunas com valores inferiores a 10 e superiores a 3
df = df[(df['PL'] < 10) & (df['PL'] > 3)]

#### Filtrar dados por P/VP inferiores a 2 e superiores a 0,5
Efetuar a filtragem dos dados com valores de P/VP (Preço sobre Valor Patrimonial) inferiores a 2 e superiores a 0,5 para evitar distorções.

In [5]:
# Filtrar linhas por colunas com valores inferiores a 2 e superiores a 0,5
df = df[(df['PVP'] < 2) & (df['PVP'] > 0.5)]

#### Filtrar Dividend Yield superiores a 7% e inferiores a 14%
Efetuar a filtragem dos dados com valores de Dividend Yield (Rendimento de Dividendos) superiores a 7% e inferiores a 14% para evitar distorções.

In [6]:
# Filtrar linhas por colunas com valores superiores a 7% e inferiores a 14%
df = df[(df['DIVYIELD'] > 7) & (df['DIVYIELD'] < 14)]

#### Ordenar dados por valores das ações

In [7]:
df = df.sort_values('COTAÇÃO')

#### Formatar valores para PT-BR (vírgulas como separadores de casas decimais)

In [8]:
df = df.map(lambda x: str(x).replace('.', ','))

#### Formatar colunas de valores percentuais para adicionar sinal de porcentagem

In [9]:
columns_to_format = ['DIVYIELD','MRG_EBIT','MRG_LÍQ','ROIC','ROE','CRESC_REC5A']
for c in columns_to_format:
    df[c] = df[c].apply(lambda x: str(x) + '%')

#### Mostrar 10 primeiras linhas

In [10]:
df.head(10)

,PAPEL,COTAÇÃO,PL,PVP,PSR,DIVYIELD,PATIVO,PCAPGIRO,PEBIT,PATIV_CIRCLIQ,...,EVEBITDA,MRG_EBIT,MRG_LÍQ,LIQ_CORR,ROIC,ROE,LIQ2MESES,PATRIM_LÍQ,DÍVBRUT_PATRIM,CRESC_REC5A
275,BMGB4,"3,43","9,77","0,51","0,0","10,95%","0,0","0,0","0,0","0,0",...,"0,0","0,0%","0,0%","0,0","0,0%","5,21%","3713410,0","3925170000,0","0,0","-6,07%"
231,JHSF3,"4,27","8,1","0,58","1,928","8,64%","0,27","1,46","6,27","-1,26",...,"10,46","30,73%","23,92%","2,85","4,52%","7,1%","12437600,0","5040500000,0","0,61","29,74%"
263,MELK3,"4,7","9,33","0,8","0,819","11,94%","0,4","0,74","10,03","1,08",...,"8,45","8,17%","12,84%","2,9","4,91%","8,62%","661124,0","1205200000,0","0,24","22,85%"
189,CIEL3,"5,15","6,7","1,14","2,163","8,45%","0,122","2,15","10,0","4,42",...,"4,13","21,63%","32,26%","1,07","19,34%","17,02%","160113000,0","12258800000,0","0,63","-8,03%"
270,PFRM3,"5,41","9,52","0,54","0,077","7,13%","0,136","0,61","2,73","4,77",...,"3,19","2,84%","0,9%","1,45","8,55%","5,64%","1154650,0","1248350000,0","0,62","16,59%"
217,BEES3,"8,86","7,55","1,26","0,0","7,43%","0,0","0,0","0,0","0,0",...,"0,0","0,0%","0,0%","0,0","0,0%","16,72%","114040,0","2216350000,0","0,0","12,8%"
235,FESA4,"8,87","8,19","0,98","1,287","7,72%","0,762","2,89","8,33","4,71",...,"4,39","15,45%","15,72%","3,17","11,92%","11,99%","6316920,0","3190100000,0","0,09","21,5%"
170,RANI3,"9,11","5,7","1,71","1,37","8,75%","0,63","2,63","4,88","-2,08",...,"5,73","28,07%","24,05%","3,75","16,34%","29,96%","12413000,0","1279700000,0","1,27","17,82%"
230,ITSA3,"10,56","8,1","1,31","14,773","8,78%","1,035","13,88","144,84","-19,32",...,"58,14","10,2%","189,33%","2,64","0,78%","16,23%","2046870,0","82952000000,0","0,13","12,11%"
234,ITSA4,"10,58","8,11","1,32","14,801","8,77%","1,037","13,91","145,12","-19,36",...,"58,24","10,2%","189,33%","2,64","0,78%","16,23%","207499000,0","82952000000,0","0,13","12,11%"


#### Carregar dados para uma planilha excel

In [11]:
df.to_excel('data.xlsx',index=False)